**Instrukcja dotyczy wybranych bibliotek powiązanych z technologią CUDA. Pełną listę przykładów znaleźć można na stronie:
https://developer.nvidia.com/gpu-accelerated-libraries**

**Na końcu instrukcji znajdują się linki do oryginalnych przykładów, na których oparta jest ta instrukcja.**

**W pierwszej kolejności usuniemy aktualnie zainstalowaną wersję oprogramowania z NVidia:**


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi




**Następnie pobierzemy i zainstalujemy oprogramowanie CUDA w wersji 9.2:**


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-11-08 11:37:38--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?zvyZuWFSy3COVmGzgq88qAqJwwlbShx1QL_SochgPivweQbiQnv8qZKL_YrFjtGP98fSQIkrkhrTgZd07ZOGRKEWg2fMHJH_HBHfB_N_JfziZ8413jfstDdpF3OI4vc9fJMN2wa7gCtKYI7wK91JEJdcRBs8GDUn1-hhWXv3_CpjaZYx_ARt_8GS9v6LOw0j2V3m8TZ2PSMK3oBEmeQ [following]
--2020-11-08 11:37:38--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?zvyZuWFSy3COVmGzgq88qAqJwwlbShx1QL_SochgPivweQbiQnv8qZKL_YrFjtGP98fSQIkrkhrTgZd07ZOGRKEWg2fMHJH_HBHfB_N_JfziZ8413jfstDdpF3

**Sprawdźmy zainstalowaną wersję nvcc:**


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88



**Konieczna jest też instalacja pluginu, który umożliwi nam pisanie kodu w języku C++ w Colab:**


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2n8j0a2e
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2n8j0a2e
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=517eb4fa2b253bb5146aa004949f09d4e08b620a0fe3dd0e832fcaf5f4a41a5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ad1bv3k3/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


**Teraz możemy wyświetlić informację o dostępnej karcie graficznej. Warto zwrócić uwagę m.in. na liczbę wątków dostępnych w bloku:**

In [ ]:
%%cu
#include <memory>
#include <iostream>
#include <cuda_runtime.h>
// Main Program
int main(void)
{
    int device_Count = 0;
    cudaGetDeviceCount(&device_Count);
    // This function returns count of number of CUDA enable devices and 0 if there are no CUDA capable devices.
    if (device_Count == 0)
    {
        printf("There are no available device(s) that support CUDA\n");
    }
    else
    {
        printf("Detected %d CUDA Capable device(s)\n", device_Count);
    }
 
    int device = 0;
    int driver_Version, runtime_Version;
 
    cudaDeviceProp device_Property;
    cudaGetDeviceProperties(&device_Property, device);
    printf("\nDevice %d: \"%s\"\n", device, device_Property.name);
    cudaDriverGetVersion(&driver_Version);
    cudaRuntimeGetVersion(&runtime_Version);
    printf(" CUDA Driver Version / Runtime Version %d.%d / %d.%d\n", driver_Version / 1000, (driver_Version % 100) / 10, runtime_Version / 1000, (runtime_Version % 100) / 10);
    printf( " Total amount of global memory: %.0f MBytes (%llu bytes)\n",
    (float)device_Property.totalGlobalMem / 1048576.0f, (unsigned long long) device_Property.totalGlobalMem);
    printf(" (%2d) Multiprocessors", device_Property.multiProcessorCount );
    printf("  GPU Max Clock rate: %.0f MHz (%0.2f GHz)\n", device_Property.clockRate * 1e-3f, device_Property.clockRate * 1e-6f);
    printf("\n");
    printf( " Total amount of global memory: %.0f MBytes (%llu bytes)\n",
    (float)device_Property.totalGlobalMem / 1048576.0f, (unsigned long long) device_Property.totalGlobalMem);
    printf(" Memory Clock rate: %.0f Mhz\n", device_Property.memoryClockRate * 1e-3f);
    printf(" Memory Bus Width: %d-bit\n", device_Property.memoryBusWidth);
    if (device_Property.l2CacheSize)
    {
        printf(" L2 Cache Size: %d bytes\n", device_Property.l2CacheSize);
    }
    printf(" Total amount of constant memory: %lu bytes\n",         device_Property.totalConstMem);
    printf(" Total amount of shared memory per block: %lu bytes\n", device_Property.sharedMemPerBlock);
    printf(" Total number of registers available per block: %d\n", device_Property.regsPerBlock);
    printf("\n");
    printf(" Maximum number of threads per multiprocessor: %d\n",              device_Property.maxThreadsPerMultiProcessor);
    printf(" Maximum number of threads per block: %d\n",         device_Property.maxThreadsPerBlock);
    printf(" Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
        device_Property.maxThreadsDim[0],
        device_Property.maxThreadsDim[1],
        device_Property.maxThreadsDim[2]);
    printf(" Max dimension size of a grid size (x,y,z): (%d, %d, %d)\n",
        device_Property.maxGridSize[0],
        device_Property.maxGridSize[1],
        device_Property.maxGridSize[2]);        
}

Detected 1 CUDA Capable device(s)

Device 0: "Tesla P100-PCIE-16GB"
 CUDA Driver Version / Runtime Version 10.1 / 9.2
 Total amount of global memory: 16281 MBytes (17071734784 bytes)
 (56) Multiprocessors  GPU Max Clock rate: 1329 MHz (1.33 GHz)

 Total amount of global memory: 16281 MBytes (17071734784 bytes)
 Memory Clock rate: 715 Mhz
 Memory Bus Width: 4096-bit
 L2 Cache Size: 4194304 bytes
 Total amount of constant memory: 65536 bytes
 Total amount of shared memory per block: 49152 bytes
 Total number of registers available per block: 65536

 Maximum number of threads per multiprocessor: 2048
 Maximum number of threads per block: 1024
 Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
 Max dimension size of a grid size (x,y,z): (2147483647, 65535, 65535)



**Pierwszy przykład będzie dotyczył wykorzystania biblioteki curand. Biblioteka ta umożliwia generowanie wartości losowych z różnych rozkładów prawdopodobieństwa. Poniższy skrypt zapisuje kod do pliku my_curand.cu:**

In [ ]:
%%cuda --name my_curand.cu 
/*
 * This program uses the host CURAND API to generate pseudorandom floats.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>
#include <chrono>
#include <random>
#include <iostream>

#define CUDA_CALL(x) do { if((x)!=cudaSuccess) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)
#define CURAND_CALL(x) do { if((x)!=CURAND_STATUS_SUCCESS) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)

int main(int argc, char *argv[])
{
    size_t n = 10000000;
    curandGenerator_t gen;
    float *devData, *hostData;

    std::chrono::steady_clock::time_point beginGPU = std::chrono::steady_clock::now();
    /* Allocate n floats on host */
    hostData = (float *)calloc(n, sizeof(float));

    /* Allocate n floats on device */
    CUDA_CALL(cudaMalloc((void **)&devData, n*sizeof(float)));

    /* Create pseudo-random number generator */
    CURAND_CALL(curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT));

    /* Set seed */
    CURAND_CALL(curandSetPseudoRandomGeneratorSeed(gen, time(NULL)));

    /* Generate n floats on device */
    //CURAND_CALL(curandGenerateUniform(gen, devData, n));
    CURAND_CALL(curandGenerateNormal(gen, devData, n, 0.0, 1.0));

    /* Copy device memory to host */
    CUDA_CALL(cudaMemcpy(hostData, devData, n * sizeof(float), cudaMemcpyDeviceToHost));

    std::chrono::steady_clock::time_point endGPU = std::chrono::steady_clock::now();
    std::cout << "Time difference GPU = " << std::chrono::duration_cast<std::chrono::microseconds>(endGPU - beginGPU).count() << "[µs]" << std::endl;
 
    /* Show result */
    /*for(int i = 0; i < n; i++) {
        printf("%1.4f ", hostData[i]);
    }
    printf("\n");*/

    /* Cleanup */
    CURAND_CALL(curandDestroyGenerator(gen));
    CUDA_CALL(cudaFree(devData));
    free(hostData);    
 
    // CPU
    std::chrono::steady_clock::time_point begin = std::chrono::steady_clock::now();
    std::default_random_engine generator;
    std::normal_distribution<double> distribution(0.0,1.0);

    std::vector<float> vectCPU(n, 0.0);
    for (int i=0; i<n; ++i) {
      vectCPU[i] = distribution(generator);
    }
    std::chrono::steady_clock::time_point end = std::chrono::steady_clock::now();
    std::cout << "Time difference CPU = " << std::chrono::duration_cast<std::chrono::microseconds>(end - begin).count() << "[µs]" << std::endl;

    return EXIT_SUCCESS;
}

'File written in /content/src/my_curand.cu'

**Teraz kompilujemy napisany kod:**

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand

**I uruchamiamy plik wykonywalny:**

In [ ]:
!/content/src/my_curand

Time difference GPU = 244452[µs]
Time difference CPU = 1094180[µs]


**Kolejny przykład dotyczy wykorzystania biblioteki cublas i csolver (algebra liniowa). Za pomocą tych bibliotek rozwiążemy równanie macierzowe Ax=B:**

In [ ]:
%%cuda --name cudenseSolver.cu 

/* 
 *  QR Factorization Dense Linear Solver
 *
 */

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#include <cuda_runtime.h>

#include <cublas_v2.h>
#include <cusolverDn.h>


void printMatrix(int m, int n, const double*A, int lda, const char* name)
{
    for(int row = 0 ; row < m ; row++){
        for(int col = 0 ; col < n ; col++){
            double Areg = A[row + col*lda];
            printf("%s(%d,%d) = %f\n", name, row+1, col+1, Areg);
        }
    } 
}

int main(int argc, char*argv[])
{
    cusolverDnHandle_t cudenseH = NULL;
    cublasHandle_t cublasH = NULL;
    cublasStatus_t cublas_status = CUBLAS_STATUS_SUCCESS;
    cusolverStatus_t cusolver_status = CUSOLVER_STATUS_SUCCESS;
    cudaError_t cudaStat1 = cudaSuccess;
    cudaError_t cudaStat2 = cudaSuccess;
    cudaError_t cudaStat3 = cudaSuccess;
    cudaError_t cudaStat4 = cudaSuccess;
    const int m = 3;
    const int lda = m;
    const int ldb = m;
    const int nrhs = 1; // number of right hand side vectors
/*       | 1 5 3 |
 *   A = | 4 5 6 | 
 *       | 2 4 1 |
 *   x = ( 1 1 1 )'
 *   b = ( 6 15 4)'
 */
    double A[lda*m] = { 1.0, 4.0, 2.0, 5.0, 5.0, 4.0, 3.0, 6.0, 1.0};
    double B[ldb*nrhs] = { 6.0, 15.0, 4.0};
    double XC[ldb*nrhs]; // solution matrix from GPU

    double *d_A = NULL; // linear memory of GPU
    double *d_tau = NULL; // linear memory of GPU
    double *d_B  = NULL;
    int *devInfo = NULL; // info in gpu (device copy)
    double *d_work = NULL;
    int  lwork = 0;

    int info_gpu = 0;

    const double one = 1;
    printf("A = (matlab base-1)\n");
    printMatrix(m, m, A, lda, "A");
    printf("=====\n");
    printf("B = (matlab base-1)\n");
    printMatrix(m, nrhs, B, ldb, "B");
    printf("=====\n");

// step 1: create cudense/cublas handle
    cusolver_status = cusolverDnCreate(&cudenseH);
    assert(CUSOLVER_STATUS_SUCCESS == cusolver_status);

    cublas_status = cublasCreate(&cublasH);
    assert(CUBLAS_STATUS_SUCCESS == cublas_status);

// step 2: copy A and B to device
    cudaStat1 = cudaMalloc ((void**)&d_A  , sizeof(double) * lda * m);
    cudaStat2 = cudaMalloc ((void**)&d_tau, sizeof(double) * m);
    cudaStat3 = cudaMalloc ((void**)&d_B  , sizeof(double) * ldb * nrhs);
    cudaStat4 = cudaMalloc ((void**)&devInfo, sizeof(int));
    assert(cudaSuccess == cudaStat1);
    assert(cudaSuccess == cudaStat2);
    assert(cudaSuccess == cudaStat3);
    assert(cudaSuccess == cudaStat4);
    cudaStat1 = cudaMemcpy(d_A, A, sizeof(double) * lda * m   , cudaMemcpyHostToDevice);
    cudaStat2 = cudaMemcpy(d_B, B, sizeof(double) * ldb * nrhs, cudaMemcpyHostToDevice);
    assert(cudaSuccess == cudaStat1);
    assert(cudaSuccess == cudaStat2);

// step 3: query working space of geqrf and ormqr
    cusolver_status = cusolverDnDgeqrf_bufferSize(cudenseH, m, m, d_A, lda, &lwork);
    assert (cusolver_status == CUSOLVER_STATUS_SUCCESS);

    cudaStat1 = cudaMalloc((void**)&d_work, sizeof(double)*lwork);
    assert(cudaSuccess == cudaStat1);

// step 4: compute QR factorization
    // Octave/Matlab: [Q,R]=qr(A)
    // inne: LU factorization (lower–upper), Cholesky, 
    cusolver_status = cusolverDnDgeqrf(cudenseH, m, m, d_A, lda, d_tau, d_work, lwork, devInfo);
    cudaStat1 = cudaDeviceSynchronize();
    assert(CUSOLVER_STATUS_SUCCESS == cusolver_status);
    assert(cudaSuccess == cudaStat1);

    // check if QR is good or not
    cudaStat1 = cudaMemcpy(&info_gpu, devInfo, sizeof(int), cudaMemcpyDeviceToHost);
    assert(cudaSuccess == cudaStat1);

    printf("after geqrf: info_gpu = %d\n", info_gpu);
    assert(0 == info_gpu);

// step 5: compute Q^T*B
    cusolver_status= cusolverDnDormqr(cudenseH, CUBLAS_SIDE_LEFT, CUBLAS_OP_T, m, nrhs, m, d_A, lda, d_tau, d_B, ldb, d_work, lwork, devInfo);
    cudaStat1 = cudaDeviceSynchronize();
    assert(CUSOLVER_STATUS_SUCCESS == cusolver_status);
    assert(cudaSuccess == cudaStat1);

    // check if QR is good or not
    cudaStat1 = cudaMemcpy(&info_gpu, devInfo, sizeof(int), cudaMemcpyDeviceToHost);
    assert(cudaSuccess == cudaStat1);

    printf("after ormqr: info_gpu = %d\n", info_gpu);
    assert(0 == info_gpu);

// step 6: compute x = R \ Q^T*B

    cublas_status = cublasDtrsm(cublasH, CUBLAS_SIDE_LEFT, CUBLAS_FILL_MODE_UPPER, CUBLAS_OP_N, CUBLAS_DIAG_NON_UNIT, m, nrhs, &one, d_A, lda, d_B, ldb);
    cudaStat1 = cudaDeviceSynchronize();
    assert(CUBLAS_STATUS_SUCCESS == cublas_status);
    assert(cudaSuccess == cudaStat1);

    cudaStat1 = cudaMemcpy(XC, d_B, sizeof(double)*ldb*nrhs, cudaMemcpyDeviceToHost);
    assert(cudaSuccess == cudaStat1);

    printf("X = (matlab base-1)\n");
    printMatrix(m, nrhs, XC, ldb, "X");

// free resources
    if (d_A    ) cudaFree(d_A);
    if (d_tau  ) cudaFree(d_tau);
    if (d_B    ) cudaFree(d_B);
    if (devInfo) cudaFree(devInfo);
    if (d_work ) cudaFree(d_work);


    if (cublasH ) cublasDestroy(cublasH);
    if (cudenseH) cusolverDnDestroy(cudenseH);

    cudaDeviceReset();

    return 0; 
}



'File written in /content/src/cudenseSolver.cu'

**Kompilujemy przykład z cuSolver:**

In [ ]:
!nvcc -o /content/src/cudenseSolver /content/src/cudenseSolver.cu -lcublas -lcusolver

**i uruchamiamy:**

In [ ]:
!/content/src/cudenseSolver

A = (matlab base-1)
A(1,1) = 1.000000
A(1,2) = 5.000000
A(1,3) = 3.000000
A(2,1) = 4.000000
A(2,2) = 5.000000
A(2,3) = 6.000000
A(3,1) = 2.000000
A(3,2) = 4.000000
A(3,3) = 1.000000
=====
B = (matlab base-1)
B(1,1) = 6.000000
B(2,1) = 15.000000
B(3,1) = 4.000000
=====
after geqrf: info_gpu = 0
after ormqr: info_gpu = 0
X = (matlab base-1)
X(1,1) = 1.307692
X(2,1) = -0.076923
X(3,1) = 1.692308


**Kolejny przykład dotyczy biblioteki cuFFT (Fast Furier Transform):**


In [ ]:

%%cuda --name cufft.cu

/* Example showing the use of CUFFT for fast 1D-convolution using FFT. */
/*https://github.com/drufat/cuda-examples/blob/master/cuda/fft.cu*/

// includes, system
#include <stdlib.h>
#include <iostream>
#include <stdio.h>
#include <string.h>
#include <math.h>
#include <chrono>

// includes, project
#include <cufft.h>

using namespace std;

// Complex data type
typedef float2 Complex;
static __device__ __host__ inline Complex ComplexAdd(Complex, Complex);
static __device__ __host__ inline Complex ComplexMul(Complex, Complex);

// Filtering functions
void Convolve(const Complex*, int, const Complex*, int, Complex*);

// Padding functions
int PadData(const Complex*, Complex**, int,
            const Complex*, Complex**, int);

////////////////////////////////////////////////////////////////////////////////
// declaration, forward
void runTest(int argc, char** argv);

// The filter size is assumed to be a number smaller than the signal size
#define FILTER_KERNEL_SIZE 11

////////////////////////////////////////////////////////////////////////////////
// Program main
////////////////////////////////////////////////////////////////////////////////
int main(int argc, char** argv)
{
    runTest(argc, argv);
    runTest(argc, argv);
    runTest(argc, argv);
}

////////////////////////////////////////////////////////////////////////////////
//! Run a simple test for CUDA
////////////////////////////////////////////////////////////////////////////////
void runTest(int argc, char** argv)
{
    int signal_size = 126;
    printf("[simpleCUFFT] is starting...\n");

    // Allocate host memory for the signal
    Complex* h_signal = (Complex*)malloc(sizeof(Complex) * signal_size);
    // Initalize the memory for the signal
    for (unsigned int i = 0; i < signal_size; ++i) {
        //h_signal[i].x = rand() / (float)RAND_MAX;
        h_signal[i].x = sin(20*(2*M_PI/signal_size)*i);
        h_signal[i].y = 0;
    }
 
    std::cout << "signal=[";
    for (int sampleNo=0;sampleNo<signal_size; sampleNo++)
        std::cout << h_signal[sampleNo].x << ", ";
    std::cout << "];\n";

    // Allocate host memory for the filter
    Complex* h_filter_kernel = (Complex*)malloc(sizeof(Complex) * FILTER_KERNEL_SIZE);
    // Initalize the memory for the filter
    for (unsigned int i = 0; i < FILTER_KERNEL_SIZE; ++i) {
        h_filter_kernel[i].x = rand() / (float)RAND_MAX;
        h_filter_kernel[i].y = 0;
    }

    // Pad signal and filter kernel
    Complex* h_padded_signal;
    Complex* h_padded_filter_kernel;
    int new_size = PadData(h_signal, &h_padded_signal, signal_size,
                           h_filter_kernel, &h_padded_filter_kernel, FILTER_KERNEL_SIZE);
    int mem_size = sizeof(Complex) * new_size;
    cout << new_size << endl;

    // Allocate device memory for signal
    Complex* d_signal;
    cudaMalloc((void**)&d_signal, mem_size);
    // Copy host memory to device
    cudaMemcpy(d_signal, h_padded_signal, mem_size,
               cudaMemcpyHostToDevice);

    // Allocate device memory for filter kernel
    Complex* d_filter_kernel;
    cudaMalloc((void**)&d_filter_kernel, mem_size);

    // Copy host memory to device
    cudaMemcpy(d_filter_kernel, h_padded_filter_kernel, mem_size,
               cudaMemcpyHostToDevice);
               
    // Start the stopwatch
    auto start = chrono::high_resolution_clock::now();

    // CUFFT plan
    cufftHandle plan;
    cufftPlan1d(&plan, new_size, CUFFT_C2R, 1);

    // CUFTT exec
    cufftExecC2R(plan, (cufftComplex *)d_signal, (cufftReal *)d_signal);
 
    // copy result to host
    Complex* res_signal = (Complex*)malloc(sizeof(Complex) * signal_size/2);
    cudaMemcpy(res_signal, d_signal, mem_size/2, cudaMemcpyDeviceToHost);
    std::cout << "outx=[";
    for (int sampleNo=0;sampleNo<signal_size/2; sampleNo++)
        std::cout << res_signal[sampleNo].x << ", ";
    std::cout << "];\n";
    std::cout << "outy=[";
    for (int sampleNo=0;sampleNo<signal_size/2; sampleNo++)
        std::cout << res_signal[sampleNo].y << ", ";
    std::cout << "];\n";
 
    std::cout << "out=[";
    for (int sampleNo=0;sampleNo<signal_size/2; sampleNo++)
        std::cout << sqrt(pow(res_signal[sampleNo].x,2.0)+pow(res_signal[sampleNo].y,2.0)) << ", ";
    std::cout << "];\n";

    auto finish = chrono::high_resolution_clock::now();    
    
    auto microseconds = chrono::duration_cast<std::chrono::microseconds>(finish-start);

    cout << "elapsed " << microseconds.count() << "us" << endl;
    //Destroy CUFFT context
    cufftDestroy(plan);

    // cleanup memory
    free(h_signal);
    free(h_filter_kernel);
    free(h_padded_signal);
    free(h_padded_filter_kernel);
    cudaFree(d_signal);
    cudaFree(d_filter_kernel);

}

// Pad data
int PadData(const Complex* signal, Complex** padded_signal, int signal_size,
            const Complex* filter_kernel, Complex** padded_filter_kernel, int filter_kernel_size)
{
    int minRadius = filter_kernel_size / 2;
    int maxRadius = filter_kernel_size - minRadius;
    int new_size = signal_size + maxRadius;

    // Pad signal
    Complex* new_data = (Complex*)malloc(sizeof(Complex) * new_size);
    memcpy(new_data +           0, signal,              signal_size * sizeof(Complex));
    memset(new_data + signal_size,      0, (new_size - signal_size) * sizeof(Complex));
    *padded_signal = new_data;

    // Pad filter
    new_data = (Complex*)malloc(sizeof(Complex) * new_size);
    memcpy(new_data +                    0, filter_kernel + minRadius,                       maxRadius * sizeof(Complex));
    memset(new_data +            maxRadius,                         0, (new_size - filter_kernel_size) * sizeof(Complex));
    memcpy(new_data + new_size - minRadius,             filter_kernel,                       minRadius * sizeof(Complex));
    *padded_filter_kernel = new_data;

    return new_size;
}

////////////////////////////////////////////////////////////////////////////////
// Filtering operations
////////////////////////////////////////////////////////////////////////////////

// Computes convolution on the host
void Convolve(const Complex* signal, int signal_size,
              const Complex* filter_kernel, int filter_kernel_size,
              Complex* filtered_signal)
{
    int minRadius = filter_kernel_size / 2;
    int maxRadius = filter_kernel_size - minRadius;
    // Loop over output element indices
    for (int i = 0; i < signal_size; ++i) {
        filtered_signal[i].x = filtered_signal[i].y = 0;
        // Loop over convolution indices
        for (int j = - maxRadius + 1; j <= minRadius; ++j) {
            int k = i + j;
            if (k >= 0 && k < signal_size)
                filtered_signal[i] = ComplexAdd(filtered_signal[i], ComplexMul(signal[k], filter_kernel[minRadius - j]));
        }
    }
}

////////////////////////////////////////////////////////////////////////////////
// Complex operations
////////////////////////////////////////////////////////////////////////////////

// Complex addition
static __device__ __host__ inline Complex ComplexAdd(Complex a, Complex b)
{
    Complex c;
    c.x = a.x + b.x;
    c.y = a.y + b.y;
    return c;
}

// Complex multiplication
static __device__ __host__ inline Complex ComplexMul(Complex a, Complex b)
{
    Complex c;
    c.x = a.x * b.x - a.y * b.y;
    c.y = a.x * b.y + a.y * b.x;
    return c;
}

'File written in /content/src/cufft.cu'

**Następnie kompilujemy kod:** 

In [ ]:
!nvcc -o /content/src/cufft /content/src/cufft.cu -lcufft

**i uruchamiamy plik wykonywalny:**

In [ ]:
!/content/src/cufft

[simpleCUFFT] is starting...
signal=[0, 0.840026, 0.911506, 0.149042, -0.749781, -0.962624, -0.294755, 0.642788, 0.992239, 0.433884, -0.521435, -0.999689, -0.56332, 0.388435, 0.984808, 0.680173, -0.246757, -0.947927, -0.781832, 0.0995678, 0.889872, 0.866025, 0.0498459, -0.811938, -0.930874, -0.198146, 0.715867, 0.974928, 0.34202, -0.603804, -0.997204, -0.478254, 0.478254, 0.997204, 0.603804, -0.34202, -0.974928, -0.715867, 0.198146, 0.930874, 0.811938, -0.0498459, -0.866025, -0.889872, -0.0995678, 0.781832, 0.947927, 0.246757, -0.680173, -0.984808, -0.388435, 0.56332, 0.999689, 0.521435, -0.433884, -0.992239, -0.642788, 0.294755, 0.962624, 0.749781, -0.149042, -0.911506, -0.840026, 4.65613e-15, 0.840026, 0.911506, 0.149042, -0.749781, -0.962624, -0.294755, 0.642788, 0.992239, 0.433884, -0.521435, -0.999689, -0.56332, 0.388435, 0.984808, 0.680173, -0.246757, -0.947927, -0.781832, 0.0995678, 0.889872, 0.866025, 0.0498459, -0.811938, -0.930874, -0.198146, 0.715867, 0.974928, 0.34202, -0.6

**Ostatni przykład dotyczy biblioteki cuDNN umożliwiającej modelowanie sieci neuronowych:**

In [ ]:
%%cuda --name cudnn.cu
#include <iomanip>
#include <iostream>
#include <cstdlib>
#include <vector>

#include <cuda.h>
#include <cudnn.h>

#define CUDA_CALL(f) { \
  cudaError_t err = (f); \
  if (err != cudaSuccess) { \
    std::cout \
        << "    Error occurred: " << err << std::endl; \
    std::exit(1); \
  } \
}

#define CUDNN_CALL(f) { \
  cudnnStatus_t err = (f); \
  if (err != CUDNN_STATUS_SUCCESS) { \
    std::cout \
        << "    Error occurred: " << err << std::endl; \
    std::exit(1); \
  } \
}

__global__ void dev_const(float *px, float k) {
  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  px[tid] = k;
}

__global__ void dev_iota(float *px) {
  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  px[tid] = tid;
}

void print(const float *data, int n, int c, int h, int w) {
  std::vector<float> buffer(1 << 20);
  CUDA_CALL(cudaMemcpy(
        buffer.data(), data,
        n * c * h * w * sizeof(float),
        cudaMemcpyDeviceToHost));
  int a = 0;
  for (int i = 0; i < n; ++i) {
    for (int j = 0; j < c; ++j) {
      std::cout << "n=" << i << ", c=" << j << ":" << std::endl;
      for (int k = 0; k < h; ++k) {
        for (int l = 0; l < w; ++l) {
          std::cout << std::setw(4) << std::right << buffer[a];
          ++a;
        }
        std::cout << std::endl;
      }
    }
  }
  std::cout << std::endl;
}

int main() {
  cudnnHandle_t cudnn;
  CUDNN_CALL(cudnnCreate(&cudnn));

  // input
  // NCHW -- batch size, the number of feature maps, the height and the width
  const int in_n = 1;
  const int in_c = 1;
  const int in_h = 5;
  const int in_w = 5;
  std::cout << "in_n: " << in_n << std::endl;
  std::cout << "in_c: " << in_c << std::endl;
  std::cout << "in_h: " << in_h << std::endl;
  std::cout << "in_w: " << in_w << std::endl;
  std::cout << std::endl;

  cudnnTensorDescriptor_t in_desc;
  CUDNN_CALL(cudnnCreateTensorDescriptor(&in_desc));
  CUDNN_CALL(cudnnSetTensor4dDescriptor(
        in_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
        in_n, in_c, in_h, in_w));

  float *in_data;
  CUDA_CALL(cudaMalloc(
        &in_data, in_n * in_c * in_h * in_w * sizeof(float)));

  // filter
  const int filt_k = 1;
  const int filt_c = 1;
  const int filt_h = 2;
  const int filt_w = 2;
  std::cout << "filt_k: " << filt_k << std::endl;
  std::cout << "filt_c: " << filt_c << std::endl;
  std::cout << "filt_h: " << filt_h << std::endl;
  std::cout << "filt_w: " << filt_w << std::endl;
  std::cout << std::endl;

  cudnnFilterDescriptor_t filt_desc;
  CUDNN_CALL(cudnnCreateFilterDescriptor(&filt_desc));
  CUDNN_CALL(cudnnSetFilter4dDescriptor(
        filt_desc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW,
        filt_k, filt_c, filt_h, filt_w));

  float *filt_data;
  CUDA_CALL(cudaMalloc(
      &filt_data, filt_k * filt_c * filt_h * filt_w * sizeof(float)));

  // convolution
  const int pad_h = 1;//padding (otoczka wokół wejścia)
  const int pad_w = 1;
  const int str_h = 1;// stride (skok dla filtra)
  const int str_w = 1;
  const int dil_h = 1;//dilation (zwiększenie receptive field)
  const int dil_w = 1;
  std::cout << "pad_h: " << pad_h << std::endl;
  std::cout << "pad_w: " << pad_w << std::endl;
  std::cout << "str_h: " << str_h << std::endl;
  std::cout << "str_w: " << str_w << std::endl;
  std::cout << "dil_h: " << dil_h << std::endl;
  std::cout << "dil_w: " << dil_w << std::endl;
  std::cout << std::endl;

  cudnnConvolutionDescriptor_t conv_desc;
  CUDNN_CALL(cudnnCreateConvolutionDescriptor(&conv_desc));
  CUDNN_CALL(cudnnSetConvolution2dDescriptor(
        conv_desc,
        pad_h, pad_w, str_h, str_w, dil_h, dil_w,
        CUDNN_CONVOLUTION, CUDNN_DATA_FLOAT));

  // output
  int out_n;
  int out_c;
  int out_h;
  int out_w;
  
  CUDNN_CALL(cudnnGetConvolution2dForwardOutputDim(
        conv_desc, in_desc, filt_desc,
        &out_n, &out_c, &out_h, &out_w));

  std::cout << "out_n: " << out_n << std::endl;
  std::cout << "out_c: " << out_c << std::endl;
  std::cout << "out_h: " << out_h << std::endl;
  std::cout << "out_w: " << out_w << std::endl;
  std::cout << std::endl;

  cudnnTensorDescriptor_t out_desc;
  CUDNN_CALL(cudnnCreateTensorDescriptor(&out_desc));
  CUDNN_CALL(cudnnSetTensor4dDescriptor(
        out_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
        out_n, out_c, out_h, out_w));

  float *out_data;
  CUDA_CALL(cudaMalloc(
        &out_data, out_n * out_c * out_h * out_w * sizeof(float)));

  // algorithm
  cudnnConvolutionFwdAlgo_t algo;
  CUDNN_CALL(cudnnGetConvolutionForwardAlgorithm(
        cudnn,
        in_desc, filt_desc, conv_desc, out_desc,
        CUDNN_CONVOLUTION_FWD_PREFER_FASTEST, 0, &algo));

  std::cout << "Convolution algorithm: " << algo << std::endl;
  std::cout << std::endl;

  // workspace
  size_t ws_size;
  CUDNN_CALL(cudnnGetConvolutionForwardWorkspaceSize(
        cudnn, in_desc, filt_desc, conv_desc, out_desc, algo, &ws_size));

  float *ws_data;
  CUDA_CALL(cudaMalloc(&ws_data, ws_size));

  std::cout << "Workspace size: " << ws_size << std::endl;
  std::cout << std::endl;

  // perform
  float alpha = 1.f;
  float beta = 0.f;
  dev_iota<<<in_w * in_h, in_n * in_c>>>(in_data);
  dev_const<<<filt_w * filt_h, filt_k * filt_c>>>(filt_data, 1.f);
  CUDNN_CALL(cudnnConvolutionForward(
      cudnn,
      &alpha, in_desc, in_data, filt_desc, filt_data,
      conv_desc, algo, ws_data, ws_size,
      &beta, out_desc, out_data));

  // results
  std::cout << "in_data:" << std::endl;
  print(in_data, in_n, in_c, in_h, in_w);
  
  std::cout << "filt_data:" << std::endl;
  print(filt_data, filt_k, filt_c, filt_h, filt_w);
  
  std::cout << "out_data:" << std::endl;
  print(out_data, out_n, out_c, out_h, out_w);

  // finalizing
  CUDA_CALL(cudaFree(ws_data));
  CUDA_CALL(cudaFree(out_data));
  CUDNN_CALL(cudnnDestroyTensorDescriptor(out_desc));
  CUDNN_CALL(cudnnDestroyConvolutionDescriptor(conv_desc));
  CUDA_CALL(cudaFree(filt_data));
  CUDNN_CALL(cudnnDestroyFilterDescriptor(filt_desc));
  CUDA_CALL(cudaFree(in_data));
  CUDNN_CALL(cudnnDestroyTensorDescriptor(in_desc));
  CUDNN_CALL(cudnnDestroy(cudnn));
  return 0;
}

'File written in /content/src/cudnn.cu'

In [ ]:
!nvcc -o /content/src/cudnn /content/src/cudnn.cu -lcudnn

In [ ]:
!/content/src/cudnn

in_n: 1
in_c: 1
in_h: 5
in_w: 5

filt_k: 1
filt_c: 1
filt_h: 2
filt_w: 2

pad_h: 1
pad_w: 1
str_h: 1
str_w: 1
dil_h: 1
dil_w: 1

out_n: 1
out_c: 1
out_h: 6
out_w: 6

Convolution algorithm: 1

Workspace size: 224

in_data:
n=0, c=0:
   0   1   2   3   4
   5   6   7   8   9
  10  11  12  13  14
  15  16  17  18  19
  20  21  22  23  24

filt_data:
n=0, c=0:
   1   1
   1   1

out_data:
n=0, c=0:
   0   1   3   5   7   4
   5  12  16  20  24  13
  15  32  36  40  44  23
  25  52  56  60  64  33
  35  72  76  80  84  43
  20  41  43  45  47  24



**Zadanie: Połączyć wybraną bibliotekę (cuFFT, cuBlas, cuSolver, cuDNN) z biblioteką cuRand i wykorzystać bibliotekę cuRand do Wygenerowania losowych danych. Kod losujący dane i wynik działania umieścić w pliku pdf.**

**Hint1: W przypadku problemów z typami float i double należy użyć odpowiedniej funkcji z biblioteki cuRand:**

- curandGenerateUniformDouble

- curandGenerateUniform // dla typu float

- curandGenerateNormalDouble

- curandGenerateNormal // dla typu float

**Hint2: Minimalna wartość jaką należy losować wynosi 10. W przypadku zadania z macierzami, proszę ustawić rozmiar macierzy A na 4x4, a wektora B na 1x4**

**Hint3: Przydatny może być kod poniżej:**



In [ ]:
%%cuda --name curand.cu 
// modified version of code from https://stackoverflow.com/questions/21112725/cuda-fill-an-matrix-with-random-values-between-a-and-b
#include <curand.h>
#include <iostream>

using namespace std;

void GPU_fill_rand(double *A, int nr_rows_A, int nr_cols_A)
{
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_XORWOW);

    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    //curandGenerateUniformDouble(prng, A, nr_rows_A * nr_cols_A);
    curandGenerateNormalDouble(prng, A, nr_rows_A * nr_cols_A, 0.0, 1.0);
}


int main(void)
{
    double   *hst_Mat , *dev_Mat;

    int Height = 4 ;
    int Width  = 4 ;
    int vSize = Height*Width ;
    int mSize = sizeof(double)*vSize ;

    hst_Mat = (double *)malloc(mSize) ;
    cudaMalloc((void**)&dev_Mat, mSize) ;

    memset(hst_Mat, 0, mSize) ;
    cudaMemset(dev_Mat, 0, mSize) ;

    GPU_fill_rand(dev_Mat, Height, Width) ;

    cudaMemcpy(hst_Mat, dev_Mat, mSize, cudaMemcpyDeviceToHost) ;

    cout << " * Result matrix : " << endl << "     " ;
    for(int i=0 ;i<Height ; i++)
    {
        for(int j=0 ; j<Width ; j++)
            cout << "   " << hst_Mat[i*Width+j] ;
            cout << endl << "     " ;
    }
    cout << endl << endl ;

    free(hst_Mat) ;
    cudaFree(dev_Mat) ;

    system("pause") ;

    return 0;
}

In [ ]:
!nvcc -o /content/src/curand /content/src/curand.cu -lcurand

In [ ]:
!/content/src/cudnn

**ACKNOWLEDGEMENTS**
- https://stackoverflow.com/questions/51194303/how-to-run-a-python-script-in-a-py-file-from-a-google-colab-notebook
- https://devblogs.nvidia.com/even-easier-introduction-cuda/
- https://devblogs.nvidia.com/unified-memory-cuda-beginners/
- https://bluewaters.ncsa.illinois.edu/liferay-content/document-library/Documentation%20Documents/test_cusolver_cuda6d5.cpp
- https://raw.githubusercontent.com/mmajko/FFT-cuda
- https://gist.github.com/odashi